leaf datasets are from tensorflow datasets and maybe swedish leaves

In [ ]:
# Imports

import numpy as np
import pathlib
import matplotlib.pyplot as plt
import IPython
import sys
import cv2
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL.Image
from PIL import ImageOps
import PIL
import datetime
import tensorboard
import sklearn
import subprocess
import pandas
import time
import glob
from IPython import display
from apiclient import discovery
import httplib2
import requests
from tensorboard.plugins.hparams import api as hp

%load_ext autoreload
%autoreload 2

from google.colab import drive             
drive.mount('/content/drive')

In [ ]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/CitrusDataset/')

# data_path_test = pathlib.Path('/content/drive/MyDrive/CarsDataset')


dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_path,
    labels= 'inferred',
    label_mode= None,
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=8,
    image_size=(128, 128),
    color_mode="rgb",
    shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
    data_path,
    labels= 'inferred',
    label_mode= None,
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=8,
    image_size=(128, 128),
    color_mode="rgb",
    shuffle=True)


# BUFFER_SIZE = 60000
BATCH_SIZE = 8
EPOCHS = 500
noise_dim = 128
num_examples_to_generate = 1

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# The generator

def make_generator_model():
    model = tf.keras.Sequential([
    # tf.keras.layers.Reshape((5, 28, 28, 1)),
    # tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1), 
    # tf.keras.layers.Normalization(axis=None, mean=0, variance=1),
    tf.keras.layers.Dense(32*32*512, use_bias=False, input_shape=(128,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Reshape((32, 32, 512)),
    # assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    tf.keras.layers.Conv2DTranspose(512, (5, 5), strides=(1, 1), padding='same', use_bias=False, kernel_initializer='glorot_normal'),
    # assert model.output_shape == (None, 7, 7, 128)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    # tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False),# kernel_initializer='glorot_normal'),
    # # assert model.output_shape == (None, 7, 7, 128)
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False, kernel_initializer='glorot_normal'),
    # assert model.output_shape == (None, 7, 7, 128)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False, kernel_initializer='glorot_normal'),
    # assert model.output_shape == (None, 14, 14, 64)
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),

    tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh', kernel_initializer='glorot_normal'),
    # assert model.output_shape == (None, 28, 28, 1)
    ])
    return model

In [ ]:
# Image generation

generator = make_generator_model()    
noise = tf.random.normal([1, 128])
generated_image = generator(noise, training=True)
plt.imshow(generated_image[0, :, :, 0])

In [ ]:
# The discriminator
# here some from tensorflow and/or pytorch tutorials others from https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/
def make_discriminator_model():
    model = tf.keras.Sequential([
    # tf.keras.layers.Normalization(axis=None, mean=0, variance=1),
    tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',input_shape=[128, 128, 3], kernel_initializer='glorot_normal'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),

    # tf.keras.layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same', kernel_initializer='glorot_normal'),
    # tf.keras.layers.LeakyReLU(),
    # tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', kernel_initializer='glorot_normal'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', kernel_initializer='glorot_normal'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', kernel_initializer='glorot_normal'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', kernel_initializer='glorot_normal'),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid'),

    # tf.keras.layers.Resizing(128, 128),
    # # tf.keras.layers.Rescaling(1./255),
    # tf.keras.layers.Conv2D(16, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    # tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP1
    # tf.keras.layers.Conv2D(32, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP2
    # tf.keras.layers.Conv2D(64, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP3
    # tf.keras.layers.Conv2D(128, 3, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.2),  
    # tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(128, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP4
    # tf.keras.layers.Dense(128, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP5
    # tf.keras.layers.Dense(2, activation='softmax')
])
    
    # model.compile(
    # optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00008),        # compiling with low learning rate
    # loss=tf.losses.BinaryCrossentropy(),
    # metrics=['accuracy'])

    return model

In [ ]:
# Decision making

discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
# Discriminator loss

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Optimizers

generator_optimizer = tf.keras.optimizers.Adam(0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)

In [ ]:
# checkpoints

checkpoint_dir = '/content/drive/MyDrive/GANCheckpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir , max_to_keep=1)

In [ ]:
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

  
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    
    if (epoch + 1) % 10 == 0:
      # manager.save()
      print("tenth is here")
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    # if (epoch + 1) % 11 == 0:
    #   service = discovery.build('drive', 'v2')  # stackoverflow stuff
    #   service.files().emptyTrash().execute()
    # Generate after the final epoch
    # display.clear_output(wait=True)
    # generate_and_save_images(generator,
    #                          epochs,
    #                          seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):

  predictions = model(test_input, training=True)

  # fig = plt.figure(figsize=(1, 1))
  for i in range(predictions.shape[0]):
      plt.imshow(predictions[i, :, :, 0])
      plt.axis('off')
  plt.savefig('image_at_epoch_{}.png'.format(epoch))
  plt.show()

In [ ]:
# plt.imshow(generated_image[0])
# plt.show()

In [ ]:
# for restoring
# if manager.latest_checkpoint:
#   checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
# Train the model
train(dataset_path, EPOCHS)

A record of changes after getting Leafish shape:

- strides on conv2dT2 from (1,1) to (2,2) DOES NOT WORK

- the dropout rates of discriminator layers now all 0.2, at 0.1 the leaf disappear

- got the generator input shape down from (32,32,512) to (32,32,256) which showed a leafish shape on the first run then vanished in the second. and keep appearing and vanishing with more epochs (big randomness in the results)

- now introducing a "resize" layer in the start of discriminator to give it (64,64,3) gives good few epochs and then vanish

- divide all the discriminator hidden unites numbers by two hope for a longer life for the gradient, and again with the vanishing so will start working on the generator because probably the dis is working by the proof of the first few epochs.

- first on the gen got first conv2dT from 512 to 256 A MESS

- to get it close from the dis model got 128-128-64-32-3 on gen. EVERYTHING IS GONE i AM NOT DOING GOOD

- mul all layer by 2 got some noise in the beginning (I am not good with gens)

- just wanted to say that I think something's size is really wrong

- now has same conv layer in gen as leafish and training for batch norm is off and first generated image.